<a href="https://colab.research.google.com/github/Karthick47v2/mock-buddy/blob/main/haar_cascade_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np

from tensorflow import keras

2022-07-07 16:22:54.048866: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dedsec/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-07-07 16:22:54.048888: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
import pyaudio
import wave

In [2]:
img_size = 96
edge_offset = 0

In [5]:
sample_rate = 44100
frames_per_buffer = 1024
no_of_channels = 1

In [6]:
model = keras.models.load_model('conv_model')

2022-07-07 16:26:44.788704: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-07 16:26:44.789177: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dedsec/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-07-07 16:26:44.789264: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dedsec/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-07-07 16:26:44.789317: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: lib

In [7]:

def draw_landmark(frame, rgb_frame):
  landmarks = model.predict(np.expand_dims(frame.reshape(img_size,img_size,1),0))

  for x_coord, y_coord in zip(landmarks[0, 0:136:2], landmarks[0, 1:136:2]):
    rgb_frame = cv2.circle(rgb_frame, tuple((int(x_coord),int(y_coord))), radius=1, color=(255,255,255), thickness=1)

  y_axis = landmarks[0, 54:62:2].mean()

  interactivity = abs(y_axis - (img_size / 2))

  abs_l = np.abs(landmarks[0, 0:16:2] - y_axis).mean()
  abs_r = np.abs(landmarks[0, 18:34:2] - y_axis).mean()

  ln = abs(abs_l - abs_r)

  threshold = 15 + 2
  if ln > threshold:
    rgb_frame = cv2.rectangle(rgb_frame, (0,0), (10,10), color=(0,0,255), thickness=-1)
  return rgb_frame, interactivity

In [54]:
def audio_func():
    #
    audio = pyaudio.PyAudio()
    stream = audio.open(format=pyaudio.paInt16, channels=no_of_channels,
                        rate=sample_rate, frames_per_buffer=frames_per_buffer, input=True)

    audio_frames = []
    i  = 0
    while i < 1000:
        print(i)
        i += 1
        audio_frames.append(stream.read(frames_per_buffer))

    stream.stop_stream()
    stream.close()
    audio.terminate()

    sound_file = wave.open("rec.wav", "wb")
    sound_file.setnchannels(no_of_channels)
    sound_file.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
    sound_file.setframerate(sample_rate)
    sound_file.writeframes(b''.join(audio_frames))
    sound_file.close()


In [8]:
def resize_w_pad(img, req_size):
  original_shape = (img.shape[1], img.shape[0])

  ratio = float(max(req_size))/max(original_shape)
  new_size = [int(x*ratio) for x in original_shape]

  img = cv2.resize(img, tuple(new_size))
  delta_w, delta_h = req_size[0] - new_size[0], req_size[1] - new_size[1]
  top, bottom = delta_h//2, delta_h-(delta_h//2)
  left, right = delta_w//2, delta_w-(delta_w//2)

  img = cv2.copyMakeBorder(img, top, bottom, left, right,
                           cv2.BORDER_CONSTANT, value=(0, 0, 0))
  return img


In [10]:
intrav = []
face_dnn = cv2.dnn.readNetFromCaffe(
    'dnn/deploy.prototxt', 'dnn/res10_300x300_ssd_iter_140000.caffemodel')

cap = cv2.VideoCapture(0)

while True:
  ret, frame = cap.read()

  (height, width) = frame.shape[:2]
  blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300,300)),1.0, (300,300), (104.0, 177.0, 123.0))
  face_dnn.setInput(blob)

  detections = face_dnn.forward()

  for i in range(0, detections.shape[2]):

    if detections[0, 0, i, 2] > 0.8:

      bbox = detections[0,0,i,3:7] * np.array([width, height, width, height])
      (start_x, start_y, end_x, end_y) = bbox.astype('int')

      frame = np.array(frame[start_y: end_y, start_x: end_x])
      g_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

      g_frame = resize_w_pad(
          g_frame, (img_size, img_size)).astype(np.uint8)

      rgb_frame = resize_w_pad(
          frame, (img_size, img_size)).astype(np.uint8)

      frame, intractiv = draw_landmark(g_frame, rgb_frame)

      intrav.append(intractiv)
      break

  # cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
  # cv2.resizeWindow('frame', 300, 300)
  cv2.imshow('frame', frame)

  # return if q pressed
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

# release webcamq
cap.release()

# close window
cv2.destroyAllWindows()


error: OpenCV(4.6.0) /io/opencv/modules/dnn/src/caffe/caffe_io.cpp:1126: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "dnn/deploy.prototxt" in function 'ReadProtoFromTextFile'


In [44]:
print(sum(intrav)/len(intrav))

5.503059579180433


In [58]:
# if __name__ == '__main__':

#     audio_cpu = mp.Process(target=audio_func)
#     video_cpu = mp.Process(target=video_func)
#     audio_cpu.start()
#     video_cpu.start()


In [9]:
# audio_func()
# video_func()

Qt: Session management error: Could not open network socket


KeyboardInterrupt: 